In [6]:
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset,LeRobotDatasetMetadata
from pathlib import Path
import numpy as np
import tqdm
import tyro

import openpi.shared.normalize as normalize
import openpi.training.config as _config
import openpi.training.data_loader as _data_loader
import openpi.transforms as transforms

class AgiBotDatasetMetadata(LeRobotDatasetMetadata):
    pass

class AgiBotDataset(LeRobotDataset):
    @classmethod
    def create(
        cls,
        repo_id: str,
        fps: int,
        features: dict,
        root: str | Path | None = None,
        robot_type: str | None = None,
        use_videos: bool = False,
        tolerance_s: float = 1e-4,
        image_writer_processes: int = 0,
        image_writer_threads: int = 0,
        video_backend: str | None = None,
        image_path: str | None = None
    ) -> "LeRobotDataset":
        """Create a agibotworld LeRobot Dataset from scratch in order to record data."""
        obj = cls.__new__(cls)
        obj.meta = AgiBotDatasetMetadata.create(
            repo_id=repo_id,
            fps=fps,
            robot_type=robot_type,
            features=features,
            root=root,
            use_videos=use_videos,
        )
        obj.repo_id = obj.meta.repo_id
        obj.root = obj.meta.root
        obj.revision = None
        obj.tolerance_s = tolerance_s
        obj.image_writer = None

        if image_writer_processes or image_writer_threads:
            obj.start_image_writer(image_writer_processes, image_writer_threads)

        # TODO(aliberts, rcadene, alexander-soare): Merge this with OnlineBuffer/DataBuffer
        obj.episode_buffer = obj.create_episode_buffer()

        obj.episodes = None
        obj.hf_dataset = obj.create_hf_dataset()
        obj.image_transforms = None
        obj.delta_timestamps = None
        obj.delta_indices = None
        obj.episode_data_index = None
        # obj.video_backend = video_backend if video_backend is not None else get_safe_default_codec()
        obj.image_path=image_path
        # obj.DEFAULT_IMAGE_PATH="{image_path}/{}"
        return obj



In [7]:
class RemoveStrings(transforms.DataTransformFn):

    def __call__(self, x: dict) -> dict:
        return {k: v for k, v in x.items() if not np.issubdtype(np.asarray(v).dtype, np.str_)}

def create_dataset(config: _config.TrainConfig, ) -> tuple[_config.DataConfig, _data_loader.Dataset]:
    data_config = config.data.create(config.assets_dirs, config.model)
    print(f'Loading dataset {data_config.repo_id}...')
    if data_config.repo_id is None:
        raise ValueError("Data config must have a repo_id")
    # out config.model
    print(f"Data config: {data_config}")
    dataset = _data_loader.create_dataset(data_config, config.model)
    dataset = _data_loader.TransformedDataset(
        dataset,
        [
            # *data_config.repack_transforms.inputs,
            # *data_config.data_transforms.inputs,
            # Remove strings since they are not supported by JAX and are not needed to compute norm stats.
            RemoveStrings(),
        ],
    )
    return data_config, dataset

In [8]:
config_name="pi0_base_agibot_lora"
config = _config.get_config(config_name)
data_config, dataset = create_dataset(config)

num_frames = len(dataset)
shuffle = False
data_loader = _data_loader.TorchDataLoader(
        dataset,
        local_batch_size=8,
        num_workers=0,
        shuffle=shuffle,
        num_batches=num_frames,
    )

keys = ["observation.states", "actions"]
stats = {key: normalize.RunningStats() for key in keys}
print(f"Computing normalization statistics for {num_frames} frames...")

Loading dataset /dataset/pack_in_the_supermarket...
Data config: DataConfig(repo_id='/dataset/pack_in_the_supermarket', asset_id='/dataset/pack_in_the_supermarket', norm_stats=None, repack_transforms=Group(inputs=[RepackTransform(structure={'state': 'observation.states', 'action': 'actions'})], outputs=()), data_transforms=Group(inputs=(AlohaInputs(action_dim=32, adapt_to_pi=False), DeltaActions(mask=(True, True, True, True, True, True, False, True, True, True, True, True, True, False))), outputs=(AbsoluteActions(mask=(True, True, True, True, True, True, False, True, True, True, True, True, True, False)), AlohaOutputs(adapt_to_pi=False))), model_transforms=Group(inputs=[InjectDefaultPrompt(prompt=None), ResizeImages(height=224, width=224), TokenizePrompt(tokenizer=<openpi.models.tokenizer.PaligemmaTokenizer object at 0x7f592ecbf390>)], outputs=()), use_quantile_norm=False, action_sequence_keys=('action',), prompt_from_task=True, local_files_only=True)
meta_data=LeRobotDatasetMetadata({

In [9]:
config.model

Pi0Config(action_dim=32, action_horizon=50, max_token_len=48, dtype='bfloat16', paligemma_variant='gemma_2b_lora', action_expert_variant='gemma_300m_lora')

In [10]:
for batch in tqdm.tqdm(data_loader, total=num_frames, desc="Computing stats"):
    # 输出batch的键
    print(f"batch.keys()={batch.keys()}")

Computing stats:   0%|                                                                                                                                                       | 0/6032 [00:00<?, ?it/s]

batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'obser

Computing stats:   1%|█                                                                                                                                            | 44/6032 [00:00<00:26, 227.12it/s]

batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'obser

Computing stats:   2%|██▋                                                                                                                                         | 117/6032 [00:00<00:18, 318.36it/s]

...

batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'obser

Computing stats: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 6031/6032 [00:15<00:00, 421.28it/s]

batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])
batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'obser

Computing stats: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6032/6032 [00:15<00:00, 399.64it/s]

batch.keys()=dict_keys(['actions', 'episode_index', 'frame_index', 'index', 'observation.images.path', 'observation.states', 'task_index', 'timestamp'])


In [11]:
for batch in tqdm.tqdm(data_loader, total=num_frames, desc="Computing stats"):
    # 输出一些batch的信息（比如他有哪些）
    # batch_add= preprocess(batch)
    # 在batch中添加新的键值对batch_add
    # batch.update(batch_add)
    for key in keys:
        values = np.asarray(batch[key][0])
        stats[key].update(values.reshape(-1, values.shape[-1]))

norm_stats = {key: stats.get_statistics() for key, stats in stats.items()}

output_path = config.assets_dirs / data_config.repo_id
print(f"Writing stats to: {output_path}")
normalize.save(output_path, norm_stats)

Computing stats: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6032/6032 [00:44<00:00, 135.27it/s]

Writing stats to: /dataset/pack_in_the_supermarket
